In [13]:
# === [0. Imports] ===
import os, sys


# Add the project root to the Python path
project_root = os.path.abspath("../..")
sys.path.append(project_root)

from pathlib import Path
import pandas as pd
from utils.model_evaluator import ModelEvaluator

# === [1. Detect project root robustly (supports nbclient + script + notebook)] ===
def get_project_root(marker_file=".git", fallback_name="real-estate-price-predictor"):
    """
    Traverse upward from the current working directory to find the project root.
    Looks for a marker (like .git) or fallback folder name.
    """
    current = Path.cwd().resolve()
    for parent in [current] + list(current.parents):
        if (parent / marker_file).exists() or fallback_name.lower() in parent.name.lower():
            return parent
    raise RuntimeError(f"❌ Could not find project root using marker '{marker_file}' or fallback '{fallback_name}'")

# === [2. Add project root to sys.path for local imports] ===
project_root = get_project_root()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Using detected project root: {project_root}")

# === [3. Local imports from utils] ===
from utils.experiment_tracker import ExperimentTracker
from utils.constants import METRICS_DB_PATH, CLEANED_DIR
from utils.model_evaluator import ModelEvaluator
from utils.model_table import ModelComparativeTable

# === [4. Load pre-study model metrics (CSV)] ===
model_pre_study_path = project_root / "data" / "ml_pre_study_metrics" / "model_metrics.csv"
if not model_pre_study_path.exists():
    raise FileNotFoundError(f"❌ File not found: {model_pre_study_path}")
print(f"✅ Found file: {model_pre_study_path}")

# === [5. Create model comparison object] ===
mcp = ModelComparativeTable()

# === [6. Display pre-study summary (from CSV)] ===
try:
    df_csv = pd.read_csv(model_pre_study_path)
    if df_csv.empty or "r2" not in df_csv.columns or df_csv["r2"].dropna().empty:
        raise ValueError("⚠️ CSV file is empty or missing valid 'r2' values.")
    mcp.display_model_summary_pre_study(model_pre_study_path)
except Exception as e:
    print("⚠️ Failed to display pre-study summary:", e)

# === [7. Display live summary (from SQLite)] ===
try:
    if not mcp.df_all_evals.empty:
        mcp.display_model_summary()
    else:
        print("⚠️ No experiment logs found in SQLite tracker.")
except Exception as e:
    print("⚠️ Failed to display model summary:", e)


Using detected project root: E:\_SoftEng\_BeCode\real-estate-price-predictor
✅ Found file: E:\_SoftEng\_BeCode\real-estate-price-predictor\data\ml_pre_study_metrics\model_metrics.csv
=== Model Evaluation Summary ===


,model,mae,rmse,r2,best,type,rank_r2,rmse/mae
0,Linear Regression (All Features),85 510.11 €,121 149.58 €,0.636600,,Linear,29,1.420000
1,Random Forest (All Features),65 705.67 €,98 940.13 €,0.757600,,Tree,24,1.510000
2,Linear Regression (Top Features),96 534.10 €,136 104.16 €,0.541300,,Linear,30,1.410000
3,Random Forest (Top Features),67 309.43 €,101 408.12 €,0.745400,,Tree,26,1.510000
4,XGBoost (All Features),66 302.09 €,97 431.35 €,0.764900,,Boosting,22,1.470000
5,XGBoost (Top RF Features),65 661.18 €,97 742.14 €,0.763400,,Boosting,23,1.490000
6,GradientBoosting (All Features),76 999.33 €,110 878.68 €,0.695600,,Other,28,1.440000
7,GradientBoosting (Top RF Features),76 517.08 €,110 878.68 €,0.696300,,Other,27,1.450000
8,LightGBM (All Features),65 548.48 €,97 417.23 €,0.765000,,Boosting,21,1.490000
9,LightGBM (Top RF Features),67 205.03 €,99 709.18 €,0.753800,,Boosting,25,1.480000



👉 Best model based on 'r2': CatBoost + Optuna CV (Top RF Features) – optimized ✓
=== Model Evaluation Summary ===


,id,timestamp,model,dataset,experiment,cleaning_version_id,mae,rmse,r2,type,rank_r2,rmse/mae,best
0,43,2025-06-29 18:34:55,Linear Regression (All Features),immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,Linear Regression (All Features),202506291834,N/A,N/A,0.723000,Linear,11,None,
1,44,2025-06-29 18:35:20,Random Forest (All Features),immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,Random Forest (All Features),202506291834,N/A,N/A,0.764200,Tree,7,None,
2,45,2025-06-29 18:35:28,Random Forest (Top 10 Features),immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,Random Forest (Top 10 Features),202506291834,N/A,N/A,0.741800,Tree,8,None,
3,46,2025-06-29 18:35:41,XGBoost CV (All Features) [TEST],immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,XGBoost Untuned (All Features) [TEST],202506291834,N/A,N/A,0.906300,Boosting,4,None,
4,47,2025-06-29 18:35:41,XGBoost CV (Top RF Features) [TEST],immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,XGBoost Untuned (Top RF Features) [TEST],202506291834,N/A,N/A,0.899800,Boosting,6,None,
5,48,2025-06-29 18:36:29,XGBoost + Optuna CV (All Features) [TEST],immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,XGBoost with Optuna (All Features) [TEST],202506291834,N/A,N/A,0.934600,Boosting,2,None,
6,49,2025-06-29 18:36:29,XGBoost + Optuna CV (Top RF Features) [TEST],immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,XGBoost with Optuna (Top RF Features) [TEST],202506291834,N/A,N/A,0.929800,Boosting,3,None,
7,50,2025-06-29 18:36:43,CatBoost (All Features),immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,CatBoost Baseline (All Features),202506291834,N/A,N/A,0.726400,Boosting,10,None,
8,51,2025-06-29 18:36:43,CatBoost (Top RF Features),immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,CatBoost Baseline (Top RF Features),202506291834,N/A,N/A,0.727400,Boosting,9,None,
9,52,2025-06-29 18:39:31,CatBoost + Optuna CV (All Features),immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,CatBoost with Optuna (All Features),202506291834,N/A,N/A,0.979200,Boosting,1,None,✓



👉 Best model based on R²: CatBoost + Optuna CV (All Features) ✓


In [20]:
import os
import sqlite3
import pandas as pd

# Trouver le chemin absolu de la base où qu’on soit
current_dir = os.getcwd()
project_root = current_dir
while not os.path.exists(os.path.join(project_root, "database", "metrics.db")):
    parent = os.path.dirname(project_root)
    if parent == project_root:
        raise FileNotFoundError("Impossible de trouver database/metrics.db dans les dossiers parents.")
    project_root = parent

db_path = os.path.join(project_root, "database", "metrics.db")
print("Base trouvée ici :", db_path)

# Lecture
conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM model_evaluations", conn)
conn.close()

# Ajoute la colonne is_test (modèles de test tout en bas)
df["is_test"] = df["model"].str.contains("TEST|test", case=False, na=False)
# Trie par: d’abord is_test==False (prod), puis r2 décroissant
df = df.sort_values(by=["is_test", "r2"], ascending=[True, False]).reset_index(drop=True)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(df)

# Lecture brute de la base
#conn = sqlite3.connect(db_path)
#df = pd.read_sql_query("SELECT * FROM model_evaluations", conn)
#conn.close()

# Ajouter une colonne 'is_test' pour le tri
df["is_test"] = df["model"].str.contains("TEST|test", case=False, na=False)

# D'abord les modèles sans 'TEST', triés par R² décroissant, puis les modèles test, triés aussi par R² décroissant
df = df.sort_values(by=["is_test", "r2"], ascending=[True, False]).reset_index(drop=True)

# Options d’affichage : tout voir
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)

# AFFICHAGE complet (toutes les colonnes, tout trié comme tu veux)
print("=== TABLEAU COMPLET (TRI PAR R², TEST EN BAS) ===")
display(df)  # ou print(df) selon ton environnement


Base trouvée ici : e:\_SoftEng\_BeCode\real-estate-price-predictor\database\metrics.db
    id            timestamp                                         model                                               dataset                                    experiment  cleaning_version_id  \
0   41  2025-06-29 18:30:56           CatBoost + Optuna CV (All Features)  immoweb_real_estate_cleaned_for_ml_20250629_1826.csv           CatBoost with Optuna (All Features)         202506291826   
1   52  2025-06-29 18:39:31           CatBoost + Optuna CV (All Features)  immoweb_real_estate_cleaned_for_ml_20250629_1834.csv           CatBoost with Optuna (All Features)         202506291834   
2   22  2025-06-29 17:49:29        CatBoost + Optuna CV (Top RF Features)  immoweb_real_estate_cleaned_for_ml_20250629_1742.csv        CatBoost with Optuna (Top RF Features)         202506291742   
3   21  2025-06-29 17:49:29           CatBoost + Optuna CV (All Features)  immoweb_real_estate_cleaned_for_ml_20250629_17

,id,timestamp,model,dataset,experiment,cleaning_version_id,mae,rmse,r2,is_test
0,41,2025-06-29 18:30:56,CatBoost + Optuna CV (All Features),immoweb_real_estate_cleaned_for_ml_20250629_1826.csv,CatBoost with Optuna (All Features),202506291826,20867.176266,28876.028355,0.980897,False
1,52,2025-06-29 18:39:31,CatBoost + Optuna CV (All Features),immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,CatBoost with Optuna (All Features),202506291834,21701.841801,30148.209880,0.979177,False
2,22,2025-06-29 17:49:29,CatBoost + Optuna CV (Top RF Features),immoweb_real_estate_cleaned_for_ml_20250629_1742.csv,CatBoost with Optuna (Top RF Features),202506291742,28785.609199,40088.000130,0.963182,False
3,21,2025-06-29 17:49:29,CatBoost + Optuna CV (All Features),immoweb_real_estate_cleaned_for_ml_20250629_1742.csv,CatBoost with Optuna (All Features),202506291742,35438.389057,49381.156058,0.944133,False
4,10,2025-06-29 16:44:52,CatBoost + Optuna CV (All Features),immoweb_real_estate_cleaned_for_ml_20250629_1641.csv,CatBoost with Optuna (All Features),202506291641,45214.335325,62794.750156,0.909661,False
5,42,2025-06-29 18:30:56,CatBoost + Optuna CV (Top RF Features),immoweb_real_estate_cleaned_for_ml_20250629_1826.csv,CatBoost with Optuna (Top RF Features),202506291826,44879.789454,63089.664714,0.908810,False
6,53,2025-06-29 18:39:31,CatBoost + Optuna CV (Top RF Features),immoweb_real_estate_cleaned_for_ml_20250629_1834.csv,CatBoost with Optuna (Top RF Features),202506291834,46390.435268,65021.643925,0.903140,False
7,11,2025-06-29 16:44:52,CatBoost + Optuna CV (Top RF Features),immoweb_real_estate_cleaned_for_ml_20250629_1641.csv,CatBoost with Optuna (Top RF Features),202506291641,46839.046978,65280.540746,0.902367,False
8,2,2025-06-29 16:42:13,Random Forest (All Features),immoweb_real_estate_cleaned_for_ml_20250629_1641.csv,Random Forest (All Features),202506291641,68324.023924,101428.245931,0.764228,False
9,13,2025-06-29 17:43:58,Random Forest (All Features),immoweb_real_estate_cleaned_for_ml_20250629_1742.csv,Random Forest (All Features),202506291742,68324.023924,101428.245931,0.764228,False


In [2]:
import sys, os
# Add the project root to the Python path
project_root = os.path.abspath("../..")
sys.path.append(project_root)
from utils.train_test_metrics_logger import TrainTestMetricsLogger

# Show summary table
logger = TrainTestMetricsLogger()
logger.display_table()

Rank,Best,timestamp,model,mae_train,rmse_train,r2_train,mae_test,rmse_test,r2_test,r2_gap,n_features,interpretation,ranking_score
1,✔,2025-07-01 19:33:27,CatBoost + Optuna CV (All Features),35.0 k€,48.1 k€,0.946919,60.5 k€,90.4 k€,0.812713,0.134206,72.000000,overfitting,-150946.113954
2,,2025-07-01 19:46:56,XGBoost + Optuna CV (All Features),38.2 k€,54.1 k€,0.932911,60.4 k€,90.8 k€,0.810866,0.122045,72.000000,overfitting,-151268.482978
3,,2025-07-01 19:56:34,XGBoost + Optuna CV (Top 30 Features),44.2 k€,62.0 k€,0.911938,62.4 k€,92.6 k€,0.803345,0.108594,30.000000,overfitting,-155034.137470
4,,2025-07-01 19:33:27,CatBoost + Optuna CV (Top RF Features),50.1 k€,70.1 k€,0.887493,62.7 k€,92.8 k€,0.802641,0.084852,30.000000,overfitting,-155490.843433
5,,2025-07-01 19:34:49,XGBoost CV (Top RF Features),22.1 k€,31.4 k€,0.977413,63.7 k€,96.3 k€,0.787667,0.189746,30.000000,overfitting,-159934.167991
6,,2025-07-01 19:34:49,XGBoost CV (All Features),20.3 k€,28.7 k€,0.981173,64.0 k€,96.2 k€,0.787700,0.193473,72.000000,overfitting,-160233.170278
7,,2025-07-01 19:09:26,CatBoost (All Features),65.2 k€,96.4 k€,0.786959,65.2 k€,96.4 k€,0.786959,0.000000,72.000000,good generalization,-161595.619596
8,,2025-07-01 19:09:26,CatBoost (Top RF Features),66.2 k€,97.2 k€,0.783307,66.2 k€,97.2 k€,0.783307,0.000000,30.000000,good generalization,-163403.150389
9,,2025-07-01 19:59:14,Random Forest (All Features),26.2 k€,39.1 k€,0.965067,68.3 k€,101.4 k€,0.764228,0.200838,72.000000,overfitting,-169750.741399
10,,2025-07-01 19:59:06,Linear Regression (All Features),63.7 k€,94.0 k€,0.797768,76.9 k€,109.9 k€,0.723019,0.074748,nan,overfitting,-186832.774354
